In [ ]:
!pip install sentence-transformers pinecone-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import pinecone
from pinecone import Pinecone, ServerlessSpec

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# 1. 데이터 불러오기 단계

In [ ]:
# 각 파일을 데이터프레임으로 읽기
paris_lodging = pd.read_csv('paris_lodging_unique.csv')
paris_tour = pd.read_csv('paris_tour_unique.csv')
osaka_lodging = pd.read_csv('osaka_lodging_unique.csv')
osaka_tour = pd.read_csv('osaka_tour_unique.csv')
newyork_lodging = pd.read_csv('newyork_lodging_unique.csv')
newyork_tour = pd.read_csv('newyork_tour_unique.csv')
bangkok_lodging = pd.read_csv('bangkok_lodging_unique.csv')
bangkok_tour = pd.read_csv('bangkok_tour_unique.csv')

In [ ]:
print('paris_lodging: ', paris_lodging.shape)
print('paris_tour: ', paris_tour.shape)

print('osaka_lodging: ', osaka_lodging.shape)
print('osaka_tour: ', osaka_tour.shape)

print('bangkok_lodging: ', bangkok_lodging.shape)
print('bangkok_tour: ', bangkok_tour.shape)

print('newyork_lodging: ', newyork_lodging.shape)
print('newyork_tour: ', newyork_tour.shape)

paris_lodging:  (189, 8)
paris_tour:  (521, 8)
osaka_lodging:  (193, 8)
osaka_tour:  (580, 8)
bangkok_lodging:  (204, 8)
bangkok_tour:  (354, 8)
newyork_lodging:  (149, 8)
newyork_tour:  (427, 8)


In [ ]:
paris_lodging.head()

,이름,주소,평점,위도,경도,리뷰,영업시간,카테고리
0,Maison le Bac Paris Apartments,"26 Rue Poissonnière, 75002 Paris, France",4.5,48.869669,2.348008,This was the perfect accommodation for my firs...,Monday: 9:00 AM – 6:00 PM\nTuesday: 9:00 AM – ...,"lodging, point_of_interest, establishment"
1,Hôtel d'Orsay,"93 Rue de Lille, 75007 Paris, France",4.7,48.860389,2.323112,Wonderful little hotel in an ideal location.\n...,Monday: Open 24 hours\nTuesday: Open 24 hours\...,"lodging, point_of_interest, establishment"
2,Hôtel Henriette,"9 Rue des Gobelins, 75013 Paris, France",4.8,48.835772,2.351317,Beautiful quirky stylish boutique hotel in a p...,Monday: Open 24 hours\nTuesday: Open 24 hours\...,"lodging, point_of_interest, establishment"
3,HÔTEL RACHEL,"1 Rue du Capitaine Soyer, 93310 Le Pré-Saint-G...",3.2,48.882678,2.403689,"Room was dirty, with nothing other than bed in...",Monday: 8:00 AM – 8:00 PM\nTuesday: 8:00 AM – ...,"travel_agency, lodging, point_of_interest, est..."
4,Maison Chomel,"15 Rue Chomel, 75007 Paris, France",4.6,48.851933,2.325329,Delightful experience overall. Kind and helpfu...,no operation time,"lodging, point_of_interest, establishment"


In [ ]:
paris_lodging.loc[0,'리뷰']

"This was the perfect accommodation for my first trip to Paris. The location suited all of my needs, and was very ideal and picturesque. The room has everything you need, even laundry and a full kitchen! The bed is comfy, and the large windows are beautiful. The only thing I would consider for some people is that there seems to be an elementary school with very vocal children all day long, which is disruptive if youre looking for quiet. Overall, I loved this apartment and will recommend it to everyone. Unforgettable stay.. Maison Le Bac made my Paris trip memorable. Location is wonderful lots of attractions nearby. Lots of bars nearby. Service was nice. We felt like home... My studio apartment was well managed. I had the most amazing stay at this flat in Paris! The location was perfect, close to all the major attractions and with easy access to public transportation. The flat itself was beautifully decorated and had all the amenities I needed for a comfortable stay. The host was also e

In [ ]:
import pandas as pd

df_list = [paris_lodging, paris_tour, osaka_lodging, osaka_tour, bangkok_lodging, bangkok_tour, newyork_lodging, newyork_tour]

for df in df_list:
    # Use apply to combine '리뷰' and '카테고리' for each row
    df['종합정보'] = df.apply(lambda row: str(row['리뷰']) + str(' ') + str(row['카테고리']), axis=1)

    # 결과 출력
    print(df[['종합정보']])

                                                  종합정보
0    This was the perfect accommodation for my firs...
1    Wonderful little hotel in an ideal location.\n...
2    Beautiful quirky stylish boutique hotel in a p...
3    Room was dirty, with nothing other than bed in...
4    Delightful experience overall. Kind and helpfu...
..                                                 ...
184  A great Airbnb, Katerina is very pleasant! lod...
185  It has no elevator, it's far from the transpor...
186  Great!\nI recommend. lodging, point_of_interes...
187  Stayed here with my wife and kids and had a gr...
188      nan lodging, point_of_interest, establishment

[189 rows x 1 columns]
                                                  종합정보
0    I seen lobster in the window and wanted it. I ...
1    The decor is simple but this was the nicest fo...
2    Dressing up your crepes every which way.  Very...
3    Absolutely perfect! We were seated without a r...
4    Living in the UK and frequently trav

In [ ]:
paris_lodging.loc[0,'종합정보']

"This was the perfect accommodation for my first trip to Paris. The location suited all of my needs, and was very ideal and picturesque. The room has everything you need, even laundry and a full kitchen! The bed is comfy, and the large windows are beautiful. The only thing I would consider for some people is that there seems to be an elementary school with very vocal children all day long, which is disruptive if youre looking for quiet. Overall, I loved this apartment and will recommend it to everyone. Unforgettable stay.. Maison Le Bac made my Paris trip memorable. Location is wonderful lots of attractions nearby. Lots of bars nearby. Service was nice. We felt like home... My studio apartment was well managed. I had the most amazing stay at this flat in Paris! The location was perfect, close to all the major attractions and with easy access to public transportation. The flat itself was beautifully decorated and had all the amenities I needed for a comfortable stay. The host was also e

In [ ]:
paris_lodging.head()

,이름,주소,평점,위도,경도,리뷰,영업시간,카테고리,종합정보
0,Maison le Bac Paris Apartments,"26 Rue Poissonnière, 75002 Paris, France",4.5,48.869669,2.348008,This was the perfect accommodation for my firs...,Monday: 9:00 AM – 6:00 PM\nTuesday: 9:00 AM – ...,"lodging, point_of_interest, establishment","{'이름': 'Maison le Bac Paris Apartments', '주소':..."
1,Hôtel d'Orsay,"93 Rue de Lille, 75007 Paris, France",4.7,48.860389,2.323112,Wonderful little hotel in an ideal location.\n...,Monday: Open 24 hours\nTuesday: Open 24 hours\...,"lodging, point_of_interest, establishment","{'이름': ""Hôtel d'Orsay"", '주소': '93 Rue de Lille..."
2,Hôtel Henriette,"9 Rue des Gobelins, 75013 Paris, France",4.8,48.835772,2.351317,Beautiful quirky stylish boutique hotel in a p...,Monday: Open 24 hours\nTuesday: Open 24 hours\...,"lodging, point_of_interest, establishment","{'이름': 'Hôtel Henriette', '주소': '9 Rue des Gob..."
3,HÔTEL RACHEL,"1 Rue du Capitaine Soyer, 93310 Le Pré-Saint-G...",3.2,48.882678,2.403689,"Room was dirty, with nothing other than bed in...",Monday: 8:00 AM – 8:00 PM\nTuesday: 8:00 AM – ...,"travel_agency, lodging, point_of_interest, est...","{'이름': 'HÔTEL RACHEL', '주소': '1 Rue du Capitai..."
4,Maison Chomel,"15 Rue Chomel, 75007 Paris, France",4.6,48.851933,2.325329,Delightful experience overall. Kind and helpfu...,no operation time,"lodging, point_of_interest, establishment","{'이름': 'Maison Chomel', '주소': '15 Rue Chomel, ..."


In [ ]:
pd.set_option('display.max_colwidth', None)
paris_tour[paris_tour['이름']=='Start Marais shopping']['리뷰']

,리뷰
352,Go my self this hat and other thing . The price were affordable even do is a very busy part of Paris . A lot of awesome places to eat and drink . Paris very good service amazing place


# 2. 임베딩 및 벡터DB 저장 단계

In [ ]:
# 1. 모델 로드 (Sentence-BERT)
from sentence_transformers import SentenceTransformer
from nltk.tokenize import sent_tokenize
import numpy as np

# multi-qa-mpnet-base-dot-v1 모델 로드
#model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1')
model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
model.get_sentence_embedding_dimension()

384

In [ ]:
# 3. Pinecone 초기화
from pinecone import Pinecone, ServerlessSpec

pinecone = Pinecone(api_key="your-api-key")

# 인덱스 생성
index_name = 'trip-index'
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=index_name,
        dimension=model.get_sentence_embedding_dimension(),   # 임베딩 차원은 임베딩 모델 따라 결정됨
        metric='cosine',
        spec=ServerlessSpec(
           cloud="aws",
           region="us-east-1")
    )

In [ ]:
#import nltk
#nltk.download('punkt')

In [ ]:
#df_list = [paris_lodging, paris_tour, osaka_lodging, osaka_tour, bangkok_lodging, bangkok_tour, newyork_lodging, newyork_tour]

# namespace 지정
namespace = 'paris_lodging'

index = pinecone.Index('trip-index')  # 인덱스 이름 지정

# 벡터 데이터베이스에 저장하는 루프
for i, row in paris_lodging.iterrows():

    # '종합정보' 열에서 NaN 값이 있는 경우 처리
    if pd.isna(row['종합정보']):
        continue

    # 리뷰를 문장 단위로 나누기
    sentences = sent_tokenize(row['종합정보'])

    # 문장별 임베딩 생성
    sentence_embeddings = model.encode(sentences)

    # 임베딩 결합 (평균값으로 결합)
    caption_embedding = np.mean(sentence_embeddings, axis=0)

    # 메타데이터 생성
    metadata = {
        "출처": "Google Places",
        "1_이름": row['이름'],
        "2_주소": row['주소'],
        "3_평점": row['평점'],
        "4_위도": row['위도'],
        "5_경도": row['경도'],
        "6_리뷰": str(row['리뷰']),
        "7_영업시간": row['영업시간'],
        "8_유형": row['카테고리']
    }

    # 벡터 데이터베이스에 임베딩 및 메타데이터 업서트
    index.upsert([{
        'id': f'places_{i}',
        'values': caption_embedding,
        'metadata': metadata
    }], namespace=namespace)

In [ ]:
#df_list = [paris_lodging, paris_tour, osaka_lodging, osaka_tour, bangkok_lodging, bangkok_tour, newyork_lodging, newyork_tour]

# namespace 지정
namespace = 'paris_tour'

index = pinecone.Index('trip-index')  # 인덱스 이름 지정

# 벡터 데이터베이스에 저장하는 루프
for i, row in paris_tour.iterrows():

    # '종합정보' 열에서 NaN 값이 있는 경우 처리
    if pd.isna(row['종합정보']):
        continue

    # 리뷰를 문장 단위로 나누기
    sentences = sent_tokenize(row['종합정보'])

    # 문장별 임베딩 생성
    sentence_embeddings = model.encode(sentences)

    # 임베딩 결합 (평균값으로 결합)
    caption_embedding = np.mean(sentence_embeddings, axis=0)

    # 메타데이터 생성
    metadata = {
        "출처": "Google Places",
        "1_이름": row['이름'],
        "2_주소": row['주소'],
        "3_평점": row['평점'],
        "4_위도": row['위도'],
        "5_경도": row['경도'],
        "6_리뷰": str(row['리뷰']),
        "7_영업시간": row['영업시간'],
        "8_유형": row['카테고리']
    }

    # 벡터 데이터베이스에 임베딩 및 메타데이터 업서트
    index.upsert([{
        'id': f'places_{i}',
        'values': caption_embedding,
        'metadata': metadata
    }], namespace=namespace)

In [ ]:
#df_list = [paris_lodging, paris_tour, osaka_lodging, osaka_tour, bangkok_lodging, bangkok_tour, newyork_lodging, newyork_tour]

# namespace 지정
namespace = 'osaka_lodging'

index = pinecone.Index('trip-index')  # 인덱스 이름 지정

# 벡터 데이터베이스에 저장하는 루프
for i, row in osaka_lodging.iterrows():

    # '종합정보' 열에서 NaN 값이 있는 경우 처리
    if pd.isna(row['종합정보']):
        continue

    # 리뷰를 문장 단위로 나누기
    sentences = sent_tokenize(row['종합정보'])

    # 문장별 임베딩 생성
    sentence_embeddings = model.encode(sentences)

    # 임베딩 결합 (평균값으로 결합)
    caption_embedding = np.mean(sentence_embeddings, axis=0)

    # 메타데이터 생성
    metadata = {
        "출처": "Google Places",
        "1_이름": row['이름'],
        "2_주소": row['주소'],
        "3_평점": row['평점'],
        "4_위도": row['위도'],
        "5_경도": row['경도'],
        "6_리뷰": str(row['리뷰']),
        "7_영업시간": row['영업시간'],
        "8_유형": row['카테고리']
    }

    # 벡터 데이터베이스에 임베딩 및 메타데이터 업서트
    index.upsert([{
        'id': f'places_{i}',
        'values': caption_embedding,
        'metadata': metadata
    }], namespace=namespace)

In [ ]:
#df_list = [paris_lodging, paris_tour, osaka_lodging, osaka_tour, bangkok_lodging, bangkok_tour, newyork_lodging, newyork_tour]

# namespace 지정
namespace = 'osaka_tour'

index = pinecone.Index('trip-index')  # 인덱스 이름 지정

# 벡터 데이터베이스에 저장하는 루프
for i, row in osaka_tour.iterrows():

    # '종합정보' 열에서 NaN 값이 있는 경우 처리
    if pd.isna(row['종합정보']):
        continue

    # 리뷰를 문장 단위로 나누기
    sentences = sent_tokenize(row['종합정보'])

    # 문장별 임베딩 생성
    sentence_embeddings = model.encode(sentences)

    # 임베딩 결합 (평균값으로 결합)
    caption_embedding = np.mean(sentence_embeddings, axis=0)

    # 메타데이터 생성
    metadata = {
        "출처": "Google Places",
        "1_이름": row['이름'],
        "2_주소": row['주소'],
        "3_평점": row['평점'],
        "4_위도": row['위도'],
        "5_경도": row['경도'],
        "6_리뷰": str(row['리뷰']),
        "7_영업시간": row['영업시간'],
        "8_유형": row['카테고리']
    }

    # 벡터 데이터베이스에 임베딩 및 메타데이터 업서트
    index.upsert([{
        'id': f'places_{i}',
        'values': caption_embedding,
        'metadata': metadata
    }], namespace=namespace)

In [ ]:
#df_list = [paris_lodging, paris_tour, osaka_lodging, osaka_tour, bangkok_lodging, bangkok_tour, newyork_lodging, newyork_tour]

# namespace 지정
namespace = 'bangkok_lodging'

index = pinecone.Index('trip-index')  # 인덱스 이름 지정

# 벡터 데이터베이스에 저장하는 루프
for i, row in bangkok_lodging.iterrows():

    # '종합정보' 열에서 NaN 값이 있는 경우 처리
    if pd.isna(row['종합정보']):
        continue

    # 리뷰를 문장 단위로 나누기
    sentences = sent_tokenize(row['종합정보'])

    # 문장별 임베딩 생성
    sentence_embeddings = model.encode(sentences)

    # 임베딩 결합 (평균값으로 결합)
    caption_embedding = np.mean(sentence_embeddings, axis=0)

    # 메타데이터 생성
    metadata = {
        "출처": "Google Places",
        "1_이름": row['이름'],
        "2_주소": row['주소'],
        "3_평점": row['평점'],
        "4_위도": row['위도'],
        "5_경도": row['경도'],
        "6_리뷰": str(row['리뷰']),
        "7_영업시간": row['영업시간'],
        "8_유형": row['카테고리']
    }

    # 벡터 데이터베이스에 임베딩 및 메타데이터 업서트
    index.upsert([{
        'id': f'places_{i}',
        'values': caption_embedding,
        'metadata': metadata
    }], namespace=namespace)

In [ ]:
#df_list = [paris_lodging, paris_tour, osaka_lodging, osaka_tour, bangkok_lodging, bangkok_tour, newyork_lodging, newyork_tour]

# namespace 지정
namespace = 'bangkok_tour'

index = pinecone.Index('trip-index')  # 인덱스 이름 지정

# 벡터 데이터베이스에 저장하는 루프
for i, row in bangkok_tour.iterrows():

    # '종합정보' 열에서 NaN 값이 있는 경우 처리
    if pd.isna(row['종합정보']):
        continue

    # 리뷰를 문장 단위로 나누기
    sentences = sent_tokenize(row['종합정보'])

    # 문장별 임베딩 생성
    sentence_embeddings = model.encode(sentences)

    # 임베딩 결합 (평균값으로 결합)
    caption_embedding = np.mean(sentence_embeddings, axis=0)

    # 메타데이터 생성
    metadata = {
        "출처": "Google Places",
        "1_이름": row['이름'],
        "2_주소": row['주소'],
        "3_평점": row['평점'],
        "4_위도": row['위도'],
        "5_경도": row['경도'],
        "6_리뷰": str(row['리뷰']),
        "7_영업시간": row['영업시간'],
        "8_유형": row['카테고리']
    }

    # 벡터 데이터베이스에 임베딩 및 메타데이터 업서트
    index.upsert([{
        'id': f'places_{i}',
        'values': caption_embedding,
        'metadata': metadata
    }], namespace=namespace)

In [ ]:
#df_list = [paris_lodging, paris_tour, osaka_lodging, osaka_tour, bangkok_lodging, bangkok_tour, newyork_lodging, newyork_tour]

# namespace 지정
namespace = 'newyork_lodging'

index = pinecone.Index('trip-index')  # 인덱스 이름 지정

# 벡터 데이터베이스에 저장하는 루프
for i, row in newyork_lodging.iterrows():

    # '종합정보' 열에서 NaN 값이 있는 경우 처리
    if pd.isna(row['종합정보']):
        continue

    # 리뷰를 문장 단위로 나누기
    sentences = sent_tokenize(row['종합정보'])

    # 문장별 임베딩 생성
    sentence_embeddings = model.encode(sentences)

    # 임베딩 결합 (평균값으로 결합)
    caption_embedding = np.mean(sentence_embeddings, axis=0)

    # 메타데이터 생성
    metadata = {
        "출처": "Google Places",
        "1_이름": row['이름'],
        "2_주소": row['주소'],
        "3_평점": row['평점'],
        "4_위도": row['위도'],
        "5_경도": row['경도'],
        "6_리뷰": str(row['리뷰']),
        "7_영업시간": row['영업시간'],
        "8_유형": row['카테고리']
    }

    # 벡터 데이터베이스에 임베딩 및 메타데이터 업서트
    index.upsert([{
        'id': f'places_{i}',
        'values': caption_embedding,
        'metadata': metadata
    }], namespace=namespace)

In [ ]:
#df_list = [paris_lodging, paris_tour, osaka_lodging, osaka_tour, bangkok_lodging, bangkok_tour, newyork_lodging, newyork_tour]

# namespace 지정
namespace = 'newyork_tour'

index = pinecone.Index('trip-index')  # 인덱스 이름 지정

# 벡터 데이터베이스에 저장하는 루프
for i, row in newyork_tour.iterrows():

    # '종합정보' 열에서 NaN 값이 있는 경우 처리
    if pd.isna(row['종합정보']):
        continue

    # 리뷰를 문장 단위로 나누기
    sentences = sent_tokenize(row['종합정보'])

    # 문장별 임베딩 생성
    sentence_embeddings = model.encode(sentences)

    # 임베딩 결합 (평균값으로 결합)
    caption_embedding = np.mean(sentence_embeddings, axis=0)

    # 메타데이터 생성
    metadata = {
        "출처": "Google Places",
        "1_이름": row['이름'],
        "2_주소": row['주소'],
        "3_평점": row['평점'],
        "4_위도": row['위도'],
        "5_경도": row['경도'],
        "6_리뷰": str(row['리뷰']),
        "7_영업시간": row['영업시간'],
        "8_유형": row['카테고리']
    }

    # 벡터 데이터베이스에 임베딩 및 메타데이터 업서트
    index.upsert([{
        'id': f'places_{i}',
        'values': caption_embedding,
        'metadata': metadata
    }], namespace=namespace)

In [ ]:
# Pinecone 상태 확인
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'bangkok_lodging': {'vector_count': 204},
                'bangkok_tour': {'vector_count': 439},
                'newyork_lodging': {'vector_count': 185},
                'newyork_tour': {'vector_count': 521},
                'osaka_lodging': {'vector_count': 209},
                'osaka_tour': {'vector_count': 580},
                'paris_lodging': {'vector_count': 189},
                'paris_tour': {'vector_count': 521}},
 'total_vector_count': 2848}

# 3. 쿼리 검색 단계
- 쿼리 input 형식 지정
- 쿼리 output 형식 지정

In [ ]:
# 검색 함수 정의

# 여행 스타일 및 동행인 기반
def search_places_style(city, companions, travel_style):
    query = f"Best places in {city} for {companions} with focus on {travel_style}."
    query_embedding = model.encode(query).tolist()
    namespace = f"{city}_tour"
    results_style= index.query(vector=query_embedding, top_k=30, namespace=namespace, include_metadata=True)
    return results_style

# 기본 유명 관광지
def search_places(city):
    query = f"The most famous tour places in {city}"
    query_embedding = model.encode(query).tolist()
    namespace = f"{city}_tour"
    results_best= index.query(vector=query_embedding, top_k=50, namespace=namespace, include_metadata=True)
    return results_best

In [ ]:
# 쿼리 벡터 생성 - 파리 / 여행일정
city = 'paris'
companions = 'parents'
travel_style = 'shopping', 'dining', 'nightlife'
results_style = search_places_style(city, companions, travel_style)
results_best = search_places(city)

In [ ]:
results_best

{'matches': [{'id': 'places_283',
              'metadata': {'1_이름': 'Tourism France Louvre',
                           '2_주소': "6 Rue de l'Amiral de Coligny, 75001 Paris, "
                                   'France',
                           '3_평점': '4',
                           '4_위도': 48.86057659999999,
                           '5_경도': 2.3409938,
                           '6_리뷰': 'We went on three different trips with '
                                   'them, a coincidence that we ended up with '
                                   'the same driver all 3 times we were told '
                                   'they have 10 drivers.\n'
                                   'Nicolas was amazing, very friendly and '
                                   'very skilled driver, specially in the '
                                   'madhouse streets of Paris. He provided '
                                   'bits of information on some of the '
                                   'locat

In [ ]:
print(len(results_style['matches']))
print(len(results_best['matches']))

30

In [ ]:
len(results_best['matches'])

50

In [ ]:
import pandas as pd

# 검색 결과를 CSV 파일로 저장하는 함수
def save_results_to_csv(results_style, results_best, file_name="search_results.csv"):
    # 결과에서 필요한 데이터를 추출

    # results_style
    data1 = []
    for result in results_style['matches']:
        row = {
            'ID': result['id'],
            'Score': result['score']
        }
        # 메타데이터가 포함되어 있을 경우 추가
        row.update(result.get('metadata', {}))
        data1.append(row)

    # results_best
    data2 = []
    for result in results_best['matches']:
        row = {
            'ID': result['id'],
            'Score': result['score']
        }
        # 메타데이터가 포함되어 있을 경우 추가
        row.update(result.get('metadata', {}))
        data2.append(row)

    # pandas DataFrame으로 변환
    df1 = pd.DataFrame(data1)
    df2 = pd.DataFrame(data2)
    df = pd.concat([df1, df2], ignore_index=True)
    #df = df.drop_duplicates(subset=['ID'])

    # CSV 파일로 저장
    df.to_csv(file_name, index=False)
    print(f"Results saved to {file_name}")

# 검색 결과를 CSV 파일로 저장
save_results_to_csv(results_style, results_best, file_name="paris_search_results.csv")

Results saved to paris_search_results.csv


In [ ]:
# 쿼리 벡터 생성 - 파리 / 여행일정
city = paris
companions = parents
travel_style = shopping, dining, nightlife

query = "ㅇ"
query_embedding = model.encode(query).tolist()

# 유사한 벡터 검색 (top_k=20로 상위 20개 결과)
search_results_tour = index.query(vector=query_embedding, top_k=20, namespace='paris_tour', include_metadata=True)

# 검색 결과 출력(유사도 점수 높은 순으로 출력)
for match in search_results_tour['matches']:
    print(f"Vector ID: {match['id']}")
    print(f"Score: {match['score']}")
    print(f"Metadatas: {match['metadata']}","\n")

In [ ]:
# 쿼리 벡터 생성 - 여행일정
query = "I want to plan a shopping-oriented trip with my partner in Paris from October 15, 2024, to October 17, 2024. What would be a good itinerary?"
query_embedding = model.encode(query).tolist()

# 유사한 벡터 검색 (top_k=20로 상위 20개 결과)
search_results_tour = index.query(vector=query_embedding, top_k=20, namespace='paris_tour', include_metadata=True)

# 검색 결과 출력(유사도 점수 높은 순으로 출력)
for match in search_results_tour['matches']:
    print(f"Vector ID: {match['id']}")
    print(f"Score: {match['score']}")
    print(f"Metadatas: {match['metadata']}","\n")

In [ ]:
# CSV 파일에 저장할 데이터를 준비
csv_data = [["ID", "Score", "Metadata"]]
for match in search_results_tour['matches']:
    csv_data.append([match['id'], match['score'], json.dumps(match['metadata'])])

# CSV 파일로 저장
with open('search_results_tour.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(csv_data)

In [ ]:
# 쿼리 벡터 생성 - 숙소 (추천한 관광지 주변으로 하는 쿼리 추가하면 좋을 듯)
query = "I am looking for luxury hotel to stay with my partner in Paris from October 15, 2024, to October 17, 2024."
query_embedding = model.encode(query).tolist()

# 유사한 벡터 검색 (top_k=20로 상위 20개 결과)
search_results_lodging = index.query(vector=query_embedding, top_k=20, namespace='paris_lodging', include_metadata=True)

# 검색 결과 출력(유사도 점수 높은 순으로 출력)
for match in search_results_lodging['matches']:
    print(f"Vector ID: {match['id']}")
    print(f"Score: {match['score']}")
    print(f"Metadatas: {match['metadata']}","\n")

Vector ID: places_76
Score: 0.65786618
Metadatas: {'1_이름': 'Hôtel Astoria - Astotel', '2_주소': '42 Rue de Moscou, 75008 Paris, France', '3_평점': '4.4', '4_위도': 48.8817094, '5_경도': 2.3229803, '6_리뷰': 'Stayed for a week in Sept 2024.\nLocation close to Metro stations.\nNeighborhood is quiet, fairly safe.\nThe rooms are small typical of hotels like this in Paris and  show sign of age but clean, bed was comfortable. Appreciate the minibar, which is replenished daily with water soda and juice.\nKudos to the breakfast crew especially Arleen and Tej, who aimed to please and kept the busy breakfast area well supplied. Front desk was friendly, especially Raphael. Maintenance Edgar was very responsive.\nBang for your buck. Will stay there again next time! Great location, great service, but the rooms are a tad small. Our room window faced a central patio and you could hear the neighbours or smell people smoking through the windows. Other than that, we really liked the hotel. Small rooms, but that i

In [ ]:
# CSV 파일에 저장할 데이터를 준비
csv_data = [["ID", "Score", "Metadata"]]
for match in search_results_lodging['matches']:
    csv_data.append([match['id'], match['score'], json.dumps(match['metadata'])])

# CSV 파일로 저장
with open('search_results_lodging.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(csv_data)